In [8]:
#Init enviroment
!python3 -m venv venv
!source venv/bin/activate
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement sys (from versions: none)
ERROR: No matching distribution found for sys
Note: you may need to restart the kernel to use updated packages.


## 1. Compiling to C using TVMC

This is the first part of the flow, here we use a python TVM interface and Micro TVM called TVMC to compile the ONNX model into an Ahead-Of-Time/Statically scheduled C function.

In [9]:
!tvmc compile mnist-12.onnx --target='c -keys=cpu -model=host' --runtime crt --executor-aot-interface-api c --executor=aot --executor-aot-unpacked-api=1 --desired-layout NCHW --output-format mlf --pass-config tir.disable_vectorize=1

[00:00:30] /home/otto/DTU/Special_F24/tvm/src/relay/transforms/convert_layout.cc:99: Warning: Desired layout(s) not specified for op: nn.max_pool2d
[00:00:30] /home/otto/DTU/Special_F24/tvm/src/relay/transforms/convert_layout.cc:99: Warning: Desired layout(s) not specified for op: nn.max_pool2d


<details>

<summary>Explenation of TVMC arguments</summary>

- **tvmc compile** *Is simply the command to invoke tvmc to compile.*
- **--target='c -keys=cpu -model=host'** *Is describing the target for TVM, as it can also target GPU and NPU.*
- **--runtime crt** *Is describing the runtime that we would like TVM to use. We use crt, as that is what the AOT executor is also targeting*
- **--executor-aot-interface-api c --executor=aot --executor-aot-unpacked-api=1** *This is the arguments that tells TVM to use the AOT scheduler*
- **--desired-layout NCHW** *An argument that decides MAC layout*
- **--output-format mlf** *An argument describing output format*
- **--pass-config tir.disable_vectorize=1** *An argument that disables the vectorize TVM pass*
</details>
</br>

This should create a folder called `module.tar` which should simply be extracted to reveal the folders containing C code.

This can be done by:

In [10]:
!tar -xvf module.tar --one-top-level
!rm module.tar

./
./codegen/
./codegen/host/
./codegen/host/include/
./codegen/host/include/tvmgen_default.h
./codegen/host/src/
./codegen/host/src/default_lib0.c
./codegen/host/src/default_lib1.c
./metadata.json
./parameters/
./parameters/default.params
./src/
./src/default.relay
./runtime/
./runtime/CMakeLists.txt
./runtime/include/
./runtime/include/checksum.h
./runtime/include/dlpack/
./runtime/include/dlpack/dlpack.h
./runtime/include/dmlc/
./runtime/include/dmlc/any.h
./runtime/include/dmlc/array_view.h
./runtime/include/dmlc/base.h
./runtime/include/dmlc/blockingconcurrentqueue.h
./runtime/include/dmlc/build_config_default.h
./runtime/include/dmlc/common.h
./runtime/include/dmlc/concurrency.h
./runtime/include/dmlc/concurrentqueue.h
./runtime/include/dmlc/config.h
./runtime/include/dmlc/data.h
./runtime/include/dmlc/endian.h
./runtime/include/dmlc/filesystem.h
./runtime/include/dmlc/input_split_shuffle.h
./runtime/include/dmlc/io.h
./runtime/include/dmlc/json.h
./runtime/include/dmlc/logging.h

## 2. Preparing for HLS

Now we have to prepare the C code for HLS, this requires some clean up and some small modifications.

In [11]:
!cp -r module/codegen/host/ ./
!rm -r module/

Here we do a couple changes to make the C code ready for HLS.

1. Clean up all the dependenciest that are not used.
2. Add lib0 to lib1 as an include.
3. Change the main function to not use pure pointers.

<details>
<summary>Manual work still to be done</summary>

After running the scripts, there is still some work to be:
- Move the constants and workspace from `defualt_lib0.c` to `defualt_lib1.c`, change naming to match the moved constants & add `#include <stdint.h>`
- Change the funciton signature to add the size of input output from the struct found in `defualt_lib0.c` (see example "/host" folder)
</details>
</br>

In [12]:
import cleanup as c

c.fix_header_file()
c.fix_lib_file_0("host/src/default_lib0.c")
c.fix_lib_file_1("host/src/default_lib1.c")

TypeError: fix_header_file() missing 1 required positional argument: 'path'

## 3. HLS using vitis

This part like the first part is very automated, it is mainly running the script [vitis_conifg](vitits_config.ini) to initiate the Vitis HLS

In [ ]:
%%bash
source /tools/Xilinx/Vitis_HLS/2023.2/settings64.sh
v++ -c --mode hls --config vitits_config.ini 


****** v++ v2023.2 (64-bit)
  **** SW Build 4026344 on 2023-10-11-15:42:10
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.
    ** Copyright 2022-2023 Advanced Micro Devices, Inc. All Rights Reserved.

Running Dispatch Server on port: 34323
INFO: [v++ 60-1548] Creating build summary session with primary output /home/otto/DTU/OTF/tvmgen_default_tvm_main_/tvmgen_default_tvm_main_.hlscompile_summary, at Thu May 30 23:18:48 2024
INFO: [v++ 82-31] Launching vitis_hls: vitis_hls -nolog -run csynth -work_dir /home/otto/DTU/OTF/tvmgen_default_tvm_main_ -config /home/otto/DTU/OTF/vitits_config.ini -cmdlineconfig /home/otto/DTU/OTF/tvmgen_default_tvm_main_/hls/config.cmdline

****** Vitis HLS - High-Level Synthesis from C, C++ and OpenCL v2023.2 (64-bit)
  **** SW Build 4023990 on Oct 11 2023
  **** IP Build 4028589 on Sat Oct 14 00:45:43 MDT 2023
  **** SharedData Build 4025554 on Tue Oct 10 17:18:54 MDT 2023
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.
    ** Copyr

Now inside the `tvmgen_default_tvm_main_` folder, there will be an HLS folder that will contain the accelerator.

## 4. Synthesis in vivado

Now the next part is to synthesise the accelerator. The accelerator also needs a way to communicate with some form of data manager, therefore there is a controller and uart interface in the rtl folder, to use as a template.

Using the `vivado_acc.tcl` file, the user can start a vivado project with the right initiation.

In [13]:
%%bash
source /tools/Xilinx/Vivado/2023.2/settings64.sh
vivado -source vivado_acc.tcl
rm ./*.log
rm ./*.jou


****** Vivado v2023.2 (64-bit)
  **** SW Build 4029153 on Fri Oct 13 20:13:54 MDT 2023
  **** IP Build 4028589 on Sat Oct 14 00:45:43 MDT 2023
  **** SharedData Build 4025554 on Tue Oct 10 17:18:54 MDT 2023
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.
    ** Copyright 2022-2023 Advanced Micro Devices, Inc. All Rights Reserved.

start_gui
INFO: [Common 17-206] Exiting Vivado at Fri May 31 00:00:58 2024...


# 5. Interfacing with the accelerator
After synthesis and programming, we can interface with the accelerator board trough the serial interface.

In [14]:
%run serial_interface/serial_interface.py

2024-05-31 00:05:48,184 appJar:WARNING [799/_checkMode]: Interactive mode is not fully tested, some features might not work.
2024-05-31 00:05:48,187 appJar:WARNING [810/_checkMode]: iPython is not fully tested, some features might not work.
